In [1]:
#Depedencies
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
api_key = config.coin_api_key
#get the current datetime to add to the file
from datetime import datetime
now = datetime.now()
injest_date = now.strftime("%m/%d/%Y %H:%M:%S")
filedate = now.strftime("%m/%d/%Y")

In [2]:
#API call to CoinMarketCap
def api_mapping_call(number_of_entries, api_key ):
  url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
  parameters = {
      'listing_status':'active',
      'limit': number_of_entries,
      'sort': 'cmc_rank'
  }
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key
  }

  session = Session()
  session.headers.update(headers)
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  return data


In [3]:
data = api_mapping_call(100, api_key)

In [4]:
#grabs section of dictionary 
cmc_data = data['data']
#puts the dict into a DF, now need to expand nested platform column
cmc_df = pd.DataFrame.from_dict(cmc_data, orient='columns')
#expanded Dataframe, when expanding duplicate columns come into play ie 2 ID columns
top_coins_df = pd.concat([cmc_df.drop(['platform'], axis= 1), cmc_df['platform'].apply(pd.Series)], axis= 1 )
#add column file name
top_coins_df["File_Name"] = "Coin_Ranking_Dim"
#Add column for injested at 
top_coins_df["injest_datetime"] = injest_date
#turn dataframe to CSV
top_coins_csv = top_coins_df.to_csv(index = False )


 - write DF to parquet format
 `trial_write_to_parquet = test.to_parquet(headers = 'none')`
 - write DF to CSV format
`dfasCSV = test.to_csv()`
 - write DF to string format
`dfAsString = test.to_string(header=False, index=False)`


In [5]:
#this works, needed to send it as text !!
#it works too, had to save it as a CSV
from azure.storage.filedatalake import DataLakeServiceClient
# install the following package 
# pip install azure-storage-file-datalake 
# Get the below details from your storage account
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = "newcontainer"
directory_name = "testingtoo"

def write_to_storage(storage_account_name,storage_account_key, container_name, directory_name,dataset,  file_name):
    """Function to write dataframe to storage. Specicify storage account name, storage account key, container name, directory name and file name.
    The fuction will check if container already exists, if it doesn't it will create a new container. If the already exisits, it will write the file to the specified container.
    Dataset must be saved as specified file type(csv,txt,parquet)
    File name must end in specificed file type (csv, txt, parquet)."""

    #convert dataset input to pandas DF
    df = pd.DataFrame([x.split(',') for x in dataset.split('\r\n')])
    #promote first row to headers
    df= df.rename(columns=df.iloc[0]).drop(df.index[0])
    

    service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=storage_account_key)
    try:
        file_system_client = service_client.create_file_system(file_system=container_name)
        dir_client = file_system_client.get_directory_client(directory_name)
        dir_client.create_directory()
        #set data to appropriate dataframe
        file = dataset
        file_client = dir_client.create_file(file_name)
        file_client.append_data(file, 0, len(file))
        file_client.flush_data(len(file))
    except  :
        #ResourceAlreadyExists
        file_system_client = service_client.get_file_system_client(file_system=container_name)
        dir_client = file_system_client.get_directory_client(directory_name)
        dir_client.create_directory()
        #set data to appropriate dataframe
        file = dataset
        file_client = dir_client.create_file(file_name)
        file_client.append_data(file, 0, len(file))
        file_client.flush_data(len(file))

    


    return df


In [6]:
dataframe = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key, container_name= container_name, directory_name= directory_name,\
    dataset = top_coins_csv, file_name= "top100coins.csv")

### -----------------------------------------------------------------------------------------------------------------------------------------

In [7]:
dataframe.reset_index()
dataframe.head()

,id,name,symbol,slug,rank,is_active,first_historical_data,last_historical_data,id,name,symbol,slug,token_address,File_Name,injest_datetime
1,1,Bitcoin,BTC,bitcoin,1,1,2013-04-28T18:47:21.000Z,2022-04-15T16:59:00.000Z,,,,,,Coin_Ranking_Dim,04/15/2022 10:00:47
2,1027,Ethereum,ETH,ethereum,2,1,2015-08-07T14:49:30.000Z,2022-04-15T16:59:00.000Z,,,,,,Coin_Ranking_Dim,04/15/2022 10:00:47
3,825,Tether,USDT,tether,3,1,2015-02-25T13:34:26.000Z,2022-04-15T16:59:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,Coin_Ranking_Dim,04/15/2022 10:00:47
4,1839,BNB,BNB,bnb,4,1,2017-07-25T04:30:05.000Z,2022-04-15T16:59:00.000Z,,,,,,Coin_Ranking_Dim,04/15/2022 10:00:47
5,3408,USD Coin,USDC,usd-coin,5,1,2018-10-08T18:49:28.000Z,2022-04-15T16:59:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,Coin_Ranking_Dim,04/15/2022 10:00:47


In [10]:
cmc_df

,id,name,symbol,slug,rank,is_active,first_historical_data,last_historical_data,platform
0,1,Bitcoin,BTC,bitcoin,1,1,2013-04-28T18:47:21.000Z,2022-04-15T16:59:00.000Z,None
1,1027,Ethereum,ETH,ethereum,2,1,2015-08-07T14:49:30.000Z,2022-04-15T16:59:00.000Z,None
2,825,Tether,USDT,tether,3,1,2015-02-25T13:34:26.000Z,2022-04-15T16:59:00.000Z,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET..."
3,1839,BNB,BNB,bnb,4,1,2017-07-25T04:30:05.000Z,2022-04-15T16:59:00.000Z,None
4,3408,USD Coin,USDC,usd-coin,5,1,2018-10-08T18:49:28.000Z,2022-04-15T16:59:00.000Z,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET..."
...,...,...,...,...,...,...,...,...,...
95,1168,Decred,DCR,decred,96,1,2016-02-10T19:59:29.000Z,2022-04-15T16:59:00.000Z,None
96,4846,Kava,KAVA,kava,97,1,2019-10-25T05:14:14.000Z,2022-04-15T16:59:00.000Z,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET..."
97,8857,Anchor Protocol,ANC,anchor-protocol,98,1,2021-03-17T15:35:09.000Z,2022-04-15T16:55:00.000Z,"{'id': 4172, 'name': 'Terra', 'symbol': 'LUNA'..."
98,2777,IoTeX,IOTX,iotex,99,1,2018-05-25T15:49:26.000Z,2022-04-15T16:59:00.000Z,None


In [ ]:
dataframe.columns

Index(['id', 'name', 'symbol', 'slug', 'rank', 'is_active',
       'first_historical_data', 'last_historical_data', 'id', 'name', 'symbol',
       'slug', 'token_address', 'File_Name', 'injest_datetime'],
      dtype='object')

In [ ]:
dataframe['File_Name'].unique()

array(['Coin_Ranking_Dim', None], dtype=object)

In [ ]:
dataframe.columns

Index(['id', 'name', 'symbol', 'slug', 'rank', 'is_active',
       'first_historical_data', 'last_historical_data', 'id', 'name', 'symbol',
       'slug', 'token_address', 'File_Name', 'injest_datetime'],
      dtype='object')

In [ ]:
trial.columns

Index(['name', 'rank', 'is_active', 'first_historical_data',
       'last_historical_data', 'name', 'token_address', 'File_Name',
       'injest_datetime'],
      dtype='object')

In [ ]:
dataframe['File_Name'].isnull()

1      False
2      False
3      False
4      False
5      False
       ...  
97     False
98     False
99     False
100    False
101     True
Name: File_Name, Length: 101, dtype: bool

In [ ]:
#reset the index
#then filter out the blank line, select columns, and write to refined
#the blank line isnull...was showing as 'None'
#dataframe[dataframe["id"] != " "]
dataframe.index.is_unique

True

In [ ]:
## Now working on refined dataset
## select necessary columns, make sure that id isn't null
##write to storage then can reference DF in another script, to pull for each ID
## also look into .format for naming of the files so that we can write multiple times 